# US Immigration Data
### Data Engineering Capstone Project

#### Project Summary
This project's goal is to further enrich US I94 immigration data with airports, temperature and demographics data for have a broader data spectrum for analysis.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import psycopg2
from sql_queries import *

In [2]:
#from pyspark.sql import SparkSession

#spark = SparkSession.builder.\
#config("spark.jars.repositories", "https://repos.spark-packages.org/").\
#config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
#enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [3]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

### Step 1: Scope the Project and Gather Data

#### Scope 
The main goal of this project is to enrich the US I94 immigration dataset with additional data, e.g. demographical and temperature data, to achieve a broader data basis for data analytics purposes.

Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Data Set 1: US I94 Immigration Data
**Source**: [https://travel.trade.gov/research/reports/i94/historical/2016.html](https://travel.trade.gov/research/reports/i94/historical/2016.html)

 This data comes from the US National Tourism and Trade Office and includes all different kinds of information about US immigrants.

In [4]:
# Read in the data here
df_immigration = pd.read_sas('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', 'sas7bdat', encoding='ISO-8859-1')
df_immigration.head(10)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,...,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247104e+10,00602,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,NaN,M,1953.0,09302016,NaN,NaN,AZ,9.247140e+10,00602,B2
7,20.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247161e+10,00602,B2
8,21.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20553.0,...,NaN,M,1970.0,09302016,NaN,NaN,AZ,9.247080e+10,00602,B2
9,22.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20562.0,...,NaN,M,1968.0,09302016,NaN,NaN,AZ,9.247849e+10,00608,B1


#### Data Set 2: Earth Surface Temperature Data
**Source**: [https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data)

 World temperature data provided by Kaggle - dataset pre-filtered for US to make upload to GitHub possible.

In [5]:
df_temperature = pd.read_csv('data/GlobalLandTemperaturesByCity_US.csv')
df_temperature.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
1,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
2,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
3,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
4,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W
5,1820-06-01,25.682,2.008,Abilene,United States,32.95N,100.53W
6,1820-07-01,26.268,1.802,Abilene,United States,32.95N,100.53W
7,1820-08-01,25.048,1.895,Abilene,United States,32.95N,100.53W
8,1820-09-01,22.435,2.216,Abilene,United States,32.95N,100.53W
9,1820-10-01,15.830,2.169,Abilene,United States,32.95N,100.53W


#### Data Set 3: U.S. City Demographic Data
**Source**: [https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)

This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000 and is derived from the US Census Bureau's 2015 American Community Survey.

In [6]:
df_demographics = pd.read_csv('data/us-cities-demographics.csv', delimiter=';')
df_demographics.head(10)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712.0,41971.0,80683,4815.0,8355.0,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629.0,56860.0,108489,3800.0,37038.0,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751.0,58077.0,109828,5204.0,16315.0,2.65,NC,Asian,11060


#### Data Set 4: Airport Codes
**Source**: [https://datahub.io/core/airport-codes#data](https://datahub.io/core/airport-codes#data)

This is a simple table of airport codes and corresponding cities.

In [7]:
df_airports = pd.read_csv('data/airport-codes_csv.csv')
df_airports.head(10)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100.0,NaN,US,US-OK,Alex,00AS,NaN,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810.0,NaN,US,US-AZ,Cordes,00AZ,NaN,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038.0,NaN,US,US-CA,Barstow,00CA,NaN,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87.0,NaN,US,US-CA,Biggs,00CL,NaN,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350.0,NaN,US,US-CA,Pine Valley,00CN,NaN,00CN,"-116.4597417, 32.7273736"


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

In [8]:
display(df_immigration.isna().sum())
df_immigration.describe()

cicid             0
i94yr             0
i94mon            0
i94cit            0
i94res            0
i94port           0
arrdate           0
i94mode         239
i94addr      152372
depdate      142457
i94bir          802
i94visa           0
count             0
dtadfile          1
visapost    1881250
occup       3088187
entdepa         238
entdepd      138429
entdepu     3095921
matflag      138429
biryear         802
dtaddto         477
gender       414269
insnum      2982605
airline       83627
admnum            0
fltno         19549
visatype          0
dtype: int64

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3.096313e+06,3096313.0,3096313.0,3.096313e+06,3.096313e+06,3.096313e+06,3.096074e+06,2.953856e+06,3.095511e+06,3.096313e+06,3096313.0,3.095511e+06,3.096313e+06
mean,3.078652e+06,2016.0,4.0,3.049069e+02,3.032838e+02,2.055985e+04,1.073690e+00,2.057395e+04,4.176761e+01,1.845393e+00,1.0,1.974232e+03,7.082885e+10
std,1.763278e+06,0.0,0.0,2.100269e+02,2.085832e+02,8.777339e+00,5.158963e-01,2.935697e+01,1.742026e+01,3.983910e-01,0.0,1.742026e+01,2.215442e+10
min,6.000000e+00,2016.0,4.0,1.010000e+02,1.010000e+02,2.054500e+04,1.000000e+00,1.517600e+04,-3.000000e+00,1.000000e+00,1.0,1.902000e+03,0.000000e+00
25%,1.577790e+06,2016.0,4.0,1.350000e+02,1.310000e+02,2.055200e+04,1.000000e+00,2.056100e+04,3.000000e+01,2.000000e+00,1.0,1.962000e+03,5.603523e+10
50%,3.103507e+06,2016.0,4.0,2.130000e+02,2.130000e+02,2.056000e+04,1.000000e+00,2.057000e+04,4.100000e+01,2.000000e+00,1.0,1.975000e+03,5.936094e+10
75%,4.654341e+06,2016.0,4.0,5.120000e+02,5.040000e+02,2.056700e+04,1.000000e+00,2.057900e+04,5.400000e+01,2.000000e+00,1.0,1.986000e+03,9.350987e+10
max,6.102785e+06,2016.0,4.0,9.990000e+02,7.600000e+02,2.057400e+04,9.000000e+00,4.542700e+04,1.140000e+02,3.000000e+00,1.0,2.019000e+03,9.991557e+10


In [9]:
display(df_temperature.isna().sum())
df_temperature.describe()

dt                                  0
AverageTemperature               2313
AverageTemperatureUncertainty    2313
City                                0
Country                             0
Latitude                            0
Longitude                           0
dtype: int64

,AverageTemperature,AverageTemperatureUncertainty
count,61408.000000,61408.000000
mean,12.541658,1.065334
std,9.533364,1.136540
min,-25.163000,0.041000
25%,5.808000,0.286000
50%,13.184000,0.499000
75%,20.220000,1.629250
max,33.740000,10.193000


In [10]:
display(df_demographics.isna().sum())
df_demographics.describe()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [11]:
display(df_airports.isna().sum())
df_airports.describe()

ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


#### Cleaning Steps
Document steps necessary to clean the data

In [12]:
def clean_airports_loc():
    with open('I94_SAS_Labels_Descriptions.SAS') as f:
        airports_loc = f.readlines()

    airports_loc = [x.strip() for x in airports_loc][302:893]
    airports_loc = [row.replace("'", '').strip().split('\t=\t') for row in airports_loc]
    airports_loc = [[row[0], *row[1].split(',')] for row in airports_loc]
    airports_loc = [row for row in airports_loc if len(row) == 3]
    df_airports_loc = pd.DataFrame({'airport_code':[ap[0] for ap in airports_loc], 'airport_city':[ap[1] for ap in airports_loc], 'airport_state':[ap[2] for ap in airports_loc]})
    return df_airports_loc    

df_airports_loc = clean_airports_loc()
df_airports_loc.head(10)

,airport_code,airport_city,airport_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


In [13]:
# Immigration data
print('Before:', df_immigration.shape)

def filter_immigration_data(df_immigration):
    # Remove non-existing airport codes from i94 immigration data
    df_immigration_filtered = df_immigration[df_immigration['i94port'].isin(df_airports_loc['airport_code'])].reset_index(drop=True)

    # Drop various cols containing many NaN values, drop rows containing NaN values
    df_immigration_filtered.drop(columns=['insnum', 'entdepu', 'occup', 'visapost'], inplace=True)
    df_immigration_filtered.dropna(inplace=True)
    return df_immigration_filtered

df_immigration_filtered = filter_immigration_data(df_immigration)

print('After:', df_immigration_filtered.shape)
df_immigration_filtered.head(10)

Before: (3096313, 28)
After: (2306497, 24)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
10,27.0,2016.0,4.0,101.0,101.0,BOS,20545.0,1.0,MA,20549.0,...,G,O,M,1958.0,04062016,M,LH,9.247876e+10,00422,B1
11,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,...,G,O,M,1960.0,04062016,F,LH,9.247890e+10,00422,B1
12,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,...,G,O,M,1954.0,09302016,M,AZ,9.250378e+10,00614,B2
13,30.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NJ,20578.0,...,G,O,M,1967.0,09302016,M,OS,9.247021e+10,00089,B2
14,31.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NY,20611.0,...,G,O,M,1973.0,09302016,M,OS,9.247129e+10,00089,B2
15,33.0,2016.0,4.0,101.0,101.0,HOU,20545.0,1.0,TX,20554.0,...,G,O,M,1963.0,09302016,F,TK,9.250930e+10,00033,B2
18,36.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20561.0,...,G,O,M,1979.0,09302016,M,TK,9.250626e+10,00001,B2
19,37.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20567.0,...,G,O,M,1967.0,09302016,F,AZ,9.247562e+10,00608,B2
20,38.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20575.0,...,G,O,M,1983.0,09302016,M,AZ,9.248609e+10,00608,B2
21,39.0,2016.0,4.0,101.0,101.0,MIA,20545.0,1.0,FL,20574.0,...,G,O,M,1951.0,09302016,F,TK,9.250766e+10,00077,B2


In [14]:
# Immigration data
print('Before:', df_immigration.shape)

def filter_immigration_data(df_immigration):
    # Remove non-existing airport codes from i94 immigration data
    df_immigration_filtered = df_immigration[df_immigration['i94port'].isin(df_airports_loc['airport_code'])].reset_index(drop=True)
    
    #df_airports_join = df_airports_filtered.merge(df_airports_loc, left_on = 'iata_code', right_on='airport_code').sort_values('iata_code')
    
    df_immigration_filtered = df_immigration_filtered.merge(df_airports_loc, left_on='i94port', right_on='airport_code')
    # Drop various cols containing many NaN values, drop rows containing NaN values
    df_immigration_filtered.drop(columns=['insnum', 'entdepu', 'occup', 'visapost', 'airport_code', 'airport_state'], inplace=True)
    df_immigration_filtered.dropna(inplace=True)
    return df_immigration_filtered

df_immigration_filtered = filter_immigration_data(df_immigration)

print('After:', df_immigration_filtered.shape)
df_immigration_filtered.head(10)

Before: (3096313, 28)
After: (2306497, 25)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype,airport_city
1,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,...,O,M,1960.0,04062016,F,LH,9.247890e+10,00422,B1,ATLANTA
2,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,...,O,M,1954.0,09302016,M,AZ,9.250378e+10,00614,B2,ATLANTA
3,30.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NJ,20578.0,...,O,M,1967.0,09302016,M,OS,9.247021e+10,00089,B2,ATLANTA
4,31.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NY,20611.0,...,O,M,1973.0,09302016,M,OS,9.247129e+10,00089,B2,ATLANTA
10,72.0,2016.0,4.0,103.0,103.0,ATL,20545.0,1.0,GA,20554.0,...,O,M,1961.0,06292016,M,LH,5.542801e+10,00444,WT,ATLANTA
11,73.0,2016.0,4.0,103.0,103.0,ATL,20545.0,1.0,GA,20634.0,...,O,M,1989.0,06292016,M,DL,5.544098e+10,00131,WT,ATLANTA
15,401.0,2016.0,4.0,103.0,103.0,ATL,20545.0,1.0,FL,20561.0,...,O,M,1981.0,06292016,M,OS,5.543016e+10,00097,WT,ATLANTA
16,621.0,2016.0,4.0,103.0,103.0,ATL,20545.0,1.0,GA,20556.0,...,O,M,1973.0,03312018,M,DL,9.248494e+10,00131,E2,ATLANTA
18,779.0,2016.0,4.0,104.0,104.0,ATL,20545.0,1.0,AL,20562.0,...,O,M,1942.0,06292016,F,DL,5.541707e+10,00083,WT,ATLANTA
19,780.0,2016.0,4.0,104.0,104.0,ATL,20545.0,1.0,CA,20553.0,...,O,M,1972.0,06292016,F,DL,5.543283e+10,00073,WT,ATLANTA


In [28]:
# Temperature data
print('Before:', df_temperature.shape)

def filter_temperature_data(df_temperature):
    # Drop rows containing NaN values
    df_temperature_filtered = df_temperature.dropna()
    df_temperature_filtered['City'] = df_temperature_filtered['City'].str.upper()
    df_temperature_filtered.drop_duplicates(subset='City', keep='last', inplace=True)
    # Move column city to the front
    df_temperature_filtered = df_temperature_filtered[ ['City'] + [ col for col in df_temperature_filtered.columns if col != 'City' ] ]
    return df_temperature_filtered

df_temperature_filtered = filter_temperature_data(df_temperature)

print('After:', df_temperature_filtered.shape)
df_temperature_filtered.head(10)

Before: (63721, 7)
After: (22, 7)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,City,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Latitude,Longitude
2324,ABILENE,2013-09-01,25.791,1.180,United States,32.95N,100.53W
5563,AKRON,2013-09-01,17.799,1.093,United States,40.99N,80.95W
7888,ALBUQUERQUE,2013-09-01,19.207,0.866,United States,34.56N,107.03W
11127,ALEXANDRIA,2013-09-01,19.643,1.050,United States,39.38N,76.99W
14366,ALLENTOWN,2013-09-01,17.408,1.048,United States,40.99N,74.56W
16691,AMARILLO,2013-09-01,24.500,1.259,United States,34.56N,101.19W
18668,ANAHEIM,2013-09-01,21.391,1.062,United States,32.95N,117.77W
20896,ANCHORAGE,2013-08-01,11.182,0.442,United States,61.88N,151.13W
24136,ANN ARBOR,2013-09-01,17.503,1.130,United States,42.59N,82.91W
26113,ANTIOCH,2013-09-01,20.471,0.826,United States,37.78N,122.03W


In [16]:
 # Airport data
print('Before:', df_airports.shape)

def filter_airports_data(df_airports):
    # Drop rows containing no value in column iata_code
    df_airports_filtered = df_airports.dropna(subset=['iata_code'])

    # Join dataframes airports and airports_loc to add city and state to airports, drop col iata_code, reduce and reorder cols
    df_airports_join = df_airports_filtered.merge(df_airports_loc, left_on = 'iata_code', right_on='airport_code').sort_values('iata_code')
    df_airports_join.drop_duplicates(subset=['iata_code'], keep=False, inplace=True)
    df_airports_join = df_airports_join[['iata_code', 'name', 'type', 'local_code', 'coordinates', 'airport_city',\
                                'elevation_ft', 'continent', 'iso_country', 'iso_region', 'municipality', 'gps_code']]
    return df_airports_join

df_airports_join = filter_airports_data(df_airports)

print('After:', df_airports_join.shape)
df_airports_join.head(10)

Before: (55075, 12)
After: (483, 12)


,iata_code,name,type,local_code,coordinates,airport_city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
94,ABE,Lehigh Valley International Airport,medium_airport,ABE,"-75.44080352783203, 40.652099609375",ABERDEEN,393.0,NaN,US,US-PA,Allentown,KABE
468,ABG,Abingdon Downs Airport,small_airport,NaN,"143.167007446, -17.616699218799997",ALBURG,NaN,OC,AU,AU-QLD,NaN,YABI
95,ABQ,Albuquerque International Sunport,large_airport,ABQ,"-106.609001, 35.040199",ALBUQUERQUE,5355.0,NaN,US,US-NM,Albuquerque,KABQ
84,ABS,Abu Simbel Airport,medium_airport,NaN,"31.611700058, 22.375999450699997",ALBURG SPRINGS,616.0,AF,EG,EG-ASN,Abu Simbel,HEBL
96,ACY,Atlantic City International Airport,medium_airport,ACY,"-74.57720184326172, 39.45759963989258",POMONA FIELD - ATLANTIC CITY,75.0,NaN,US,US-NJ,Atlantic City,KACY
98,ADS,Addison Airport,small_airport,ADS,"-96.8364028931, 32.9686012268",ADDISON AIRPORT- ADDISON,644.0,NaN,US,US-TX,Dallas,KADS
97,ADT,Ada Regional Airport,small_airport,ADH,"-96.671303, 34.804298",AMISTAD DAM,1016.0,NaN,US,US-OK,Ada,KADH
99,ADW,Joint Base Andrews,large_airport,ADW,"-76.866997, 38.810799",ANDREWS AFB,280.0,NaN,US,US-MD,Camp Springs,KADW
100,AFW,Fort Worth Alliance Airport,large_airport,AFW,"-97.31880187990001, 32.9875984192",FORT WORTH ALLIANCE,722.0,NaN,US,US-TX,Fort Worth,KAFW
79,AGA,Al Massira Airport,medium_airport,NaN,"-9.413069725036621, 30.325000762939453",AGANA,250.0,AF,MA,MA-AGD,Agadir,GMAD


In [74]:
 # Demographics data
print('Before:', df_demographics.shape)

def filter_demographics_data(df_demographics):
    # Drop rows containing no value in column iata_code
    df_demographics_filtered = df_demographics.dropna()
    df_demographics_filtered['City'] = df_demographics_filtered['City'].str.upper()
    df_demographics_filtered.drop_duplicates(subset='City', keep='last', inplace=True)

    return df_demographics_filtered

df_demographics_filtered = filter_demographics_data(df_demographics)

print('After:', df_demographics_filtered.shape)
df_demographics_filtered.head(10)

Before: (2891, 12)
After: (559, 12)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
660,NORTH LITTLE ROCK,Arkansas,33.6,31671.0,34835.0,66506,4130.0,2787.0,2.62,AR,American Indian and Alaska Native,713
702,LAS CRUCES,New Mexico,32.7,47835.0,53809.0,101644,9421.0,11888.0,2.58,NM,Black or African-American,3825
806,SAN BERNARDINO,California,28.5,108671.0,107466.0,216137,7741.0,55135.0,3.55,CA,Hispanic or Latino,136214
817,CHAMPAIGN,Illinois,28.7,43326.0,42760.0,86086,3734.0,12261.0,2.25,IL,Asian,11746
819,BAYONNE,New Jersey,39.7,32705.0,33598.0,66303,2225.0,21899.0,2.62,NJ,Hispanic or Latino,19525
986,HOMESTEAD,Florida,31.4,31401.0,35099.0,66500,2287.0,24408.0,3.48,FL,Hispanic or Latino,41910
992,GREELEY,Colorado,31.0,50792.0,50091.0,100883,4294.0,11480.0,2.75,CO,American Indian and Alaska Native,2449
1006,WICHITA,Kansas,34.6,192354.0,197601.0,389955,23978.0,40270.0,2.56,KS,Black or African-American,52592
1071,OMAHA,Nebraska,34.2,218789.0,225098.0,443887,24503.0,48263.0,2.47,NE,American Indian and Alaska Native,6318
1209,SPARKS,Nevada,36.1,47780.0,48318.0,96098,7315.0,15690.0,2.63,NV,White,78737


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
I decided to model the data warehouse using a star schema, as it mainly serves for OLAP and BI purposes. The I94 immigration data is the central fact table, all additional data sources are dimension tables related to the fact table.

| Table name | Columns | Description | Table type |
| ------- | ---------- | ----------- | ---- |
| immigration | cicid - year - month - cit - res - iata - arrdate - mode - addr - depdate - bir - visa - coun- dtadfil -  entdepa - entdepd - entdepu - matflag - biryear - dtaddto - gender - airline - admnum - fltno - visatype | I94 immigrations data | Fact table |
| airports | iata_code - name - type - local_code - coordinates - city | Information related to airports | Dimension table |
| demographics | city - state - media_age - male_population - female_population - total_population - num_veterans - foreign_born - average_household_size - state_code - race - count | Demographics data for cities | Dimension table |
| temperature | timestamp - average_temperature - average_temperatur_uncertainty - city - country - latitude - longitude | Temperature information | Dimension table |


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Execute create_tables.py
2. Join tables: airports - airports_loc
3. Insert data

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [76]:
!python create_tables.py

In [77]:
# Establish connection to database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [78]:
df_immigration_filtered.head(10)
df_temperature_filtered.head(10)
df_airports_join.head(10)
#df_demographics_filtered.head(10)

,iata_code,name,type,local_code,coordinates,airport_city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
94,ABE,Lehigh Valley International Airport,medium_airport,ABE,"-75.44080352783203, 40.652099609375",ABERDEEN,393.0,NaN,US,US-PA,Allentown,KABE
468,ABG,Abingdon Downs Airport,small_airport,NaN,"143.167007446, -17.616699218799997",ALBURG,NaN,OC,AU,AU-QLD,NaN,YABI
95,ABQ,Albuquerque International Sunport,large_airport,ABQ,"-106.609001, 35.040199",ALBUQUERQUE,5355.0,NaN,US,US-NM,Albuquerque,KABQ
84,ABS,Abu Simbel Airport,medium_airport,NaN,"31.611700058, 22.375999450699997",ALBURG SPRINGS,616.0,AF,EG,EG-ASN,Abu Simbel,HEBL
96,ACY,Atlantic City International Airport,medium_airport,ACY,"-74.57720184326172, 39.45759963989258",POMONA FIELD - ATLANTIC CITY,75.0,NaN,US,US-NJ,Atlantic City,KACY
98,ADS,Addison Airport,small_airport,ADS,"-96.8364028931, 32.9686012268",ADDISON AIRPORT- ADDISON,644.0,NaN,US,US-TX,Dallas,KADS
97,ADT,Ada Regional Airport,small_airport,ADH,"-96.671303, 34.804298",AMISTAD DAM,1016.0,NaN,US,US-OK,Ada,KADH
99,ADW,Joint Base Andrews,large_airport,ADW,"-76.866997, 38.810799",ANDREWS AFB,280.0,NaN,US,US-MD,Camp Springs,KADW
100,AFW,Fort Worth Alliance Airport,large_airport,AFW,"-97.31880187990001, 32.9875984192",FORT WORTH ALLIANCE,722.0,NaN,US,US-TX,Fort Worth,KAFW
79,AGA,Al Massira Airport,medium_airport,NaN,"-9.413069725036621, 30.325000762939453",AGANA,250.0,AF,MA,MA-AGD,Agadir,GMAD


In [79]:
# Insert rows in airport table
for index, row in df_airports_join.iterrows():
    cur.execute(airport_insert, list(row.values))
    conn.commit()

In [80]:
# Insert rows in immigration table
for index, row in df_immigration_filtered.head(1000).iterrows():
    cur.execute(immigration_insert, list(row.values))
    conn.commit()

In [81]:
# Insert rows in temperature table
for index, row in df_temperature_filtered.head(1000).iterrows():
    cur.execute(temperature_insert, list(row.values))
    conn.commit()

In [82]:
# Insert rows in demographics table
for index, row in df_demographics_filtered.head(1000).iterrows():
    cur.execute(demographics_insert, list(row.values))
    conn.commit()

In [83]:
# Show that data has been inserted into database tables
tables = ['airports', 'immigration', 'temperature', 'demographics']

for table in tables:
    query = f'SELECT * FROM {table} LIMIT 10'
    query_result = pd.io.sql.read_sql(query, conn)
    display(query_result)

,iata_code,name,type,local_code,coordinates,city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
0,ABE,Lehigh Valley International Airport,medium_airport,ABE,"-75.44080352783203, 40.652099609375",ABERDEEN,393.0,NaN,US,US-PA,Allentown,KABE
1,ABG,Abingdon Downs Airport,small_airport,NaN,"143.167007446, -17.616699218799997",ALBURG,NaN,OC,AU,AU-QLD,NaN,YABI
2,ABQ,Albuquerque International Sunport,large_airport,ABQ,"-106.609001, 35.040199",ALBUQUERQUE,5355.0,NaN,US,US-NM,Albuquerque,KABQ
3,ABS,Abu Simbel Airport,medium_airport,NaN,"31.611700058, 22.375999450699997",ALBURG SPRINGS,616.0,AF,EG,EG-ASN,Abu Simbel,HEBL
4,ACY,Atlantic City International Airport,medium_airport,ACY,"-74.57720184326172, 39.45759963989258",POMONA FIELD - ATLANTIC CITY,75.0,NaN,US,US-NJ,Atlantic City,KACY
5,ADS,Addison Airport,small_airport,ADS,"-96.8364028931, 32.9686012268",ADDISON AIRPORT- ADDISON,644.0,NaN,US,US-TX,Dallas,KADS
6,ADT,Ada Regional Airport,small_airport,ADH,"-96.671303, 34.804298",AMISTAD DAM,1016.0,NaN,US,US-OK,Ada,KADH
7,ADW,Joint Base Andrews,large_airport,ADW,"-76.866997, 38.810799",ANDREWS AFB,280.0,NaN,US,US-MD,Camp Springs,KADW
8,AFW,Fort Worth Alliance Airport,large_airport,AFW,"-97.31880187990001, 32.9875984192",FORT WORTH ALLIANCE,722.0,NaN,US,US-TX,Fort Worth,KAFW
9,AGA,Al Massira Airport,medium_airport,NaN,"-9.413069725036621, 30.325000762939453",AGANA,250.0,AF,MA,MA-AGD,Agadir,GMAD


,cicid,year,month,cit,res,iata,arrdate,mode,addr,depdate,...,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype,city
0,28,2016,4,101,101,ATL,20545,1.0,MA,20549,...,O,M,1960.0,04062016,F,LH,9.247890e+10,00422,B1,ATLANTA
1,29,2016,4,101,101,ATL,20545,1.0,MA,20561,...,O,M,1954.0,09302016,M,AZ,9.250378e+10,00614,B2,ATLANTA
2,30,2016,4,101,101,ATL,20545,1.0,NJ,20578,...,O,M,1967.0,09302016,M,OS,9.247021e+10,00089,B2,ATLANTA
3,31,2016,4,101,101,ATL,20545,1.0,NY,20611,...,O,M,1973.0,09302016,M,OS,9.247129e+10,00089,B2,ATLANTA
4,72,2016,4,103,103,ATL,20545,1.0,GA,20554,...,O,M,1961.0,06292016,M,LH,5.542801e+10,00444,WT,ATLANTA
5,73,2016,4,103,103,ATL,20545,1.0,GA,20634,...,O,M,1989.0,06292016,M,DL,5.544098e+10,00131,WT,ATLANTA
6,401,2016,4,103,103,ATL,20545,1.0,FL,20561,...,O,M,1981.0,06292016,M,OS,5.543016e+10,00097,WT,ATLANTA
7,621,2016,4,103,103,ATL,20545,1.0,GA,20556,...,O,M,1973.0,03312018,M,DL,9.248494e+10,00131,E2,ATLANTA
8,779,2016,4,104,104,ATL,20545,1.0,AL,20562,...,O,M,1942.0,06292016,F,DL,5.541707e+10,00083,WT,ATLANTA
9,780,2016,4,104,104,ATL,20545,1.0,CA,20553,...,O,M,1972.0,06292016,F,DL,5.543283e+10,00073,WT,ATLANTA


,city,timestamp,avg_temperature,avg_temperature_uncertainty,country,latitude,longitude
0,ABILENE,2013-09-01,25.791,1.180,United States,32.95N,100.53W
1,AKRON,2013-09-01,17.799,1.093,United States,40.99N,80.95W
2,ALBUQUERQUE,2013-09-01,19.207,0.866,United States,34.56N,107.03W
3,ALEXANDRIA,2013-09-01,19.643,1.050,United States,39.38N,76.99W
4,ALLENTOWN,2013-09-01,17.408,1.048,United States,40.99N,74.56W
5,AMARILLO,2013-09-01,24.500,1.259,United States,34.56N,101.19W
6,ANAHEIM,2013-09-01,21.391,1.062,United States,32.95N,117.77W
7,ANCHORAGE,2013-08-01,11.182,0.442,United States,61.88N,151.13W
8,ANN ARBOR,2013-09-01,17.503,1.130,United States,42.59N,82.91W
9,ANTIOCH,2013-09-01,20.471,0.826,United States,37.78N,122.03W


,city,state,media_age,male_population,female_population,total_population,num_veterans,foreign_born,average_household_size,state_code,race,count
0,NORTH LITTLE ROCK,Arkansas,33.6,31671,34835,66506,4130,2787,2.62,AR,American Indian and Alaska Native,713
1,LAS CRUCES,New Mexico,32.7,47835,53809,101644,9421,11888,2.58,NM,Black or African-American,3825
2,SAN BERNARDINO,California,28.5,108671,107466,216137,7741,55135,3.55,CA,Hispanic or Latino,136214
3,CHAMPAIGN,Illinois,28.7,43326,42760,86086,3734,12261,2.25,IL,Asian,11746
4,BAYONNE,New Jersey,39.7,32705,33598,66303,2225,21899,2.62,NJ,Hispanic or Latino,19525
5,HOMESTEAD,Florida,31.4,31401,35099,66500,2287,24408,3.48,FL,Hispanic or Latino,41910
6,GREELEY,Colorado,31.0,50792,50091,100883,4294,11480,2.75,CO,American Indian and Alaska Native,2449
7,WICHITA,Kansas,34.6,192354,197601,389955,23978,40270,2.56,KS,Black or African-American,52592
8,OMAHA,Nebraska,34.2,218789,225098,443887,24503,48263,2.47,NE,American Indian and Alaska Native,6318
9,SPARKS,Nevada,36.1,47780,48318,96098,7315,15690,2.63,NV,White,78737


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [84]:
# Check if data has been inserted into tables
tables = ['airports', 'immigration', 'temperature', 'demographics']
for table in tables:
    cur.execute(f'SELECT COUNT(*) FROM {table}')
    conn.commit()
    if cur.rowcount == 0:
        print(f'No data in table {table}')
    else:
        print(f'Data found in table {table}')

Data found in table airports
Data found in table immigration
Data found in table temperature
Data found in table demographics


In [85]:
# Check specific columns in tables for null values
table_cols = {
    'airports': ['iata_code'],
    'immigration': ['cicid'],
    'temperature': ['city'],
    'demographics': ['city']
}

for table in table_cols:
    for col in table_cols[table]:
        cur.execute(f'SELECT COUNT(*) FROM {table} WHERE {col} IS NULL')
        if cur.fetchall() == [(0,)]:
            print(f'No null values found in table {table}, column {col}.')
        else:
            print(f'Null values found in table {table}, column {col}.')

No null values found in table airports, column iata_code.
No null values found in table immigration, column cicid.
No null values found in table temperature, column city.
No null values found in table demographics, column city.


In [87]:
# Sample query to show evidence that the final data model works
query = """
    SELECT *
    FROM immigration imm
    JOIN airports air
        ON imm.iata = air.iata_code
    LIMIT 10
"""



query_result = pd.io.sql.read_sql(query, conn)
query_result

#import pandas as pd
#import pandas.io.sql as sqlio
#import psycopg2

#conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))
#sql = "select count(*) from table;"
#dat = sqlio.read_sql_query(sql, conn)
#conn = None

,cicid,year,month,cit,res,iata,arrdate,mode,addr,depdate,...,type,local_code,coordinates,city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
0,28,2016,4,101,101,ATL,20545,1.0,MA,20549,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
1,29,2016,4,101,101,ATL,20545,1.0,MA,20561,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
2,30,2016,4,101,101,ATL,20545,1.0,NJ,20578,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
3,31,2016,4,101,101,ATL,20545,1.0,NY,20611,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
4,72,2016,4,103,103,ATL,20545,1.0,GA,20554,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
5,73,2016,4,103,103,ATL,20545,1.0,GA,20634,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
6,401,2016,4,103,103,ATL,20545,1.0,FL,20561,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
7,621,2016,4,103,103,ATL,20545,1.0,GA,20556,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
8,779,2016,4,104,104,ATL,20545,1.0,AL,20562,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL
9,780,2016,4,104,104,ATL,20545,1.0,CA,20553,...,large_airport,ATL,"-84.428101, 33.6367",ATLANTA,1026.0,NaN,US,US-GA,Atlanta,KATL


In [88]:
# Close database connection
conn.close()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

**Facts table "immigration"**:

| Columns | Description |
| ------- | ---------- |
| cicid | INT PRIMARY KEY |
| year | INT |
| month | INT |
| cit | INT |
| res | INT |
| iata | VARCHAR, FK airports |
| arrdate | INT |
| mode | INT |
| addr | VARCHAR |
| depdate | INT |
| bir | FLOAT |
| visa | FLOAT |
| count | FLOAT |
| dtadfile | VARCHAR |
| entdepa | VARCHAR |
| entdepd | VARCHAR |
| matflag | VARCHAR |
| biryear | FLOAT |
| dtaddto | VARCHAR |
| gender | VARCHAR |
| airline | VARCHAR |
| admnum | FLOAT |
| fltno | VARCHAR |
| visatype | VARCHAR |
| city | VARCHAR, FK demographics, FK temperature |

**Dimensions table "airports"**:

| Columns | Description |
| ------- | ---------- |
| iata_code | VARCHAR PRIMARY KEY |
| name | VARCHAR |
| type | VARCHAR |
| local_code | VARCHAR |
| coordinates | VARCHAR |
| city | VARCHAR |
| elevation_ft | FLOAT |
| continent | VARCHAR |
| iso_country | VARCHAR |
| iso_region | VARCHAR |
| municipality | VARCHAR |
| gps_code | VARCHAR |

**Dimensions table "demographics"**:

| Columns | Description |
| ------- | ---------- |
| city | VARCHAR PRIMARY KEY |
| state | VARCHAR |
| media_age | FLOAT |
| male_population | INT |
| female_population | INT |
| total_population | INT |
| num_veterans | INT |
| foreign_born | INT |
| average_household_size | FLOAT |
| state_code | VARCHAR |
| race | VARCHAR |
| count | INT |

**Dimensions table "temperature"**:

| Columns | Description |
| ------- | ---------- |
| city | VARCHAR PRIMARY KEY |
| timestamp | DATE |
| avg_temperature | FLOAT |
| avg_temperature_uncertainty | FLOAT |
| country | VARCHAR |
| latitude | VARCHAR |
| longitude | VARCHAR |

#### Step 5: Complete Project Write Up
* **Clearly state the rationale for the choice of tools and technologies for the project.**
  
  **Python**: The programming language of (my) choice when it comes to data engineering / science projects.
  **Pandas**: Used for easy data processing.
  **PostgreSQL**: Used for non-distributed database processing on a single computer. Easy to use the same SQL query code base when switching a project to e.g. AWS Redshift, as Redshift uses PostgreSQL underneath.


* **Propose how often the data should be updated and why.**

  **Immigration data**: Aggregated on a monthly basis, hence a monthly update makes sense.
  
  
* **Write a description of how you would approach the problem differently under the following scenarios:**

  **1. The data was increased by 100x.**
  
  In this case it would make sense to use Spark for distributed processing of the data and switching to a cloud solution such as AWS EMR.
  
  **2. The data populates a dashboard that must be updated on a daily basis by 7am every day.**
  
  I would use Airflow to create a DAG to automatize these updates on a daily basis.
  
  **3. The database needed to be accessed by 100+ people.**
  
  I would recommend to switch the database to Redshift for scalable access via e.g. the frontend of a data analytics platform such as Tableau.